<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L04_1_Implicit_Feedback_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 43.6 MB/s eta 0:00:00


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-04-12 13:30:36--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2024-04-12 13:30:36 (6.57 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

rows = []
cols = []

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())
    for line in f:
        uid, mid, _, _ = line.split(",")
        rows.append(int(uid))
        cols.append(int(mid))

rows = np.array(rows)
cols = np.array(cols)
data = np.array([1] * len(rows))

ratings_csr = csr_matrix((data, (rows, cols)))

userId,movieId,rating,timestamp



In [ ]:
from implicit import bpr

# train
model = bpr.BayesianPersonalizedRanking(factors = 10)
model.fit(ratings_csr)

# test
users = [2, 3]
ids, scores = model.recommend(users, ratings_csr[users])
print(ids)
print(scores)


  0%|          | 0/100 [00:00<?, ?it/s]

[[112556  68954 106920  80463 122904  84152 112852  59315 109374  72998]
 [  1081   1959    501   4713   3706   2366   2352   2243   2436   2926]]
[[3.7234542 3.5628586 3.5373905 3.5190513 3.4725142 3.3585346 3.3403656
  3.3343365 3.3256602 3.312197 ]
 [2.4529557 2.4329557 2.410814  2.3336844 2.293305  2.261212  2.23107
  2.230677  2.213483  2.2097297]]


# BPR with Pytorch

In [ ]:
import torch

users = torch.from_numpy(rows)
items = torch.from_numpy(cols)

n_factors = 10
n_items = max(items) + 1
n_users = max(users) + 1

item_bias = torch.randn(n_items, requires_grad=True)
item_factor = torch.randn(n_items, n_factors, requires_grad=True)
user_factor = torch.randn(n_users, n_factors, requires_grad=True)

In [ ]:
lmd = 0.01
n_ratings = len(users)

optim = torch.optim.Adam([item_bias, item_factor, user_factor], lr=0.1)

logsigmoid = torch.nn.LogSigmoid()

for epoch in range(100):

    neg_items = torch.randint(1, n_items, (n_ratings,))

    pos_score = item_bias[items] + (user_factor[users] * item_factor[items]).sum(dim=1)
    neg_score = item_bias[neg_items] + (user_factor[users] * item_factor[neg_items]).sum(dim=1)
    reg = (item_bias ** 2).sum() + (item_factor ** 2).sum() + (user_factor ** 2).sum()

    cost = -logsigmoid(pos_score - neg_score).sum() + lmd * reg

    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        train_acc = sum(pos_score > neg_score) / n_ratings
        print(f"epoch: {epoch}, train accuracy: {train_acc.item()}, cost: {cost.item()}")

In [ ]:
with torch.no_grad():

    n_outputs = 10

    uid = 2

    scores_all = item_bias + (user_factor[uid] * item_factor).sum(dim=1)
    scores_all = scores_all.numpy()

    ids = np.argsort(scores_all)[::-1]
    pure_ids = ids[np.isin(ids, items[users == uid], invert=True)]

    pure_ids = pure_ids[:n_outputs]
    scores = scores_all[pure_ids]

    print(pure_ids)
    print(scores)
